# GOAL: find and remove data of the wrong type in our data frames

Written by Brian Mattis 4/5/2022. https://medium.com/@brian-mattis

char in numeric columns, numbers in char columns

In [3]:
import numpy as np
import pandas as pd
import random

## Data Creation

In [55]:
df = pd.DataFrame({'num_col'  : np.tile([3, 5.46, 62.1, np.nan, False, 'angie'], 7),
                   'name_col' : np.tile(['bob', 'sue', 'steve', 24.5, 'kim', True, np.nan], 6)})

In [5]:
df.head(7)

,num_col,name_col
0,3,bob
1,5.46,sue
2,62.1,steve
3,nan,24.5
4,False,kim
5,angie,True
6,3,nan


In [6]:
df.dtypes

num_col     object
name_col    object
dtype: object

Note - object type can't detect our 'nan' entries.. so can't get those cleaned up until we address the other bad data

In [56]:
df.isnull().sum()

num_col     0
name_col    0
dtype: int64

## Remove Chars and Booleans from a Numeric column (but keep NaNs for later review)

In [7]:
#functionalized.  usage: char_finder(df, 'num_col')
def char_finder(data_frame, series_name):
    cnt=0
    print(series_name)
    for row in data_frame[series_name]:
        try:
            float(row) #changed to float to not flag NaNs or decimals.  
            pass
        except ValueError:
            print(data_frame.loc[cnt, series_name], "-> at row:"+str(cnt))
        cnt+=1

In [6]:
char_finder(df, 'num_col')

num_col
False -> at row:4
angie -> at row:5
False -> at row:10
angie -> at row:11
False -> at row:16
angie -> at row:17
False -> at row:22
angie -> at row:23
False -> at row:28
angie -> at row:29
False -> at row:34
angie -> at row:35
False -> at row:40
angie -> at row:41


In [7]:
df.dtypes['num_col']

dtype('O')

In [94]:
#functionalized.  usage: char_finder(df, 'num_col')
def char_fixer(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            float(row) #changed to float to not flag NaNs or decimals.  
            pass
        except ValueError:
            data_frame.drop([cnt], inplace=True) #or assign to some other desired value
        cnt+=1
    data_frame[series_name] = data_frame[series_name].astype('float64', errors='raise')
    data_frame.reset_index(drop=True, inplace=True)

In [9]:
char_fixer(df,'num_col')

Let's check to see what type our column is now

In [10]:
df.dtypes['num_col']

dtype('float64')

We can use the finder function to do one last check to make sure the characters are removed

In [11]:
char_finder(df, 'num_col')

num_col


Make a version wheere all bad data is converted to NaN, not dropped.

In [9]:
#functionalized.  usage: char_finder(df, 'num_col')
def char_fixer_nan(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            float(row) #changed to float to not flag NaNs or decimals.  
            pass
        except ValueError:
            data_frame.loc[cnt, series_name] = np.nan
        cnt+=1
    data_frame[series_name] = data_frame[series_name].astype('float64', errors='raise')

In [10]:
char_fixer_nan(df,'num_col')

In [11]:
df

,num_col,name_col
0,3.00,bob
1,5.46,sue
2,62.10,steve
3,NaN,24.5
4,NaN,kim
...,...,...
37,5.46,steve
38,62.10,24.5
39,NaN,kim
40,NaN,True


## Remove numbers and booleans from a character column

In [99]:
df = pd.DataFrame({'num_col'  : np.tile([3, 5.46, 62.1, np.nan, False, 'angie'], 7),
                   'name_col' : np.tile(['bob', 'sue', 'steve', 24.5, 'kim', True, np.nan], 6)})

Keep in mind that Pandas cast booleans to strings in the dataframe conversion.

In [100]:
df['name_col'][5]

'True'

In [101]:
isinstance(df['name_col'][5], bool)

False

Therefore, float() will not be able to separate the booleans from the character values, and we'll have to deal with them separately

In [102]:
#functionalized.  usage: num_finder(df, 'name_col')
def num_finder(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            int(float(row)) # if no error, report found numeric in bottom 'else:'
        except ValueError:
            if row=='True' or row=='False':
                # report the True or False entries
                print(data_frame.loc[cnt, series_name], "-> at row:"+str(cnt))
            else: #Chars or NaNs
                pass
        else:
            print(data_frame.loc[cnt, series_name], "-> at row:"+str(cnt))
        cnt+=1

In [103]:
num_finder(df,'name_col')

24.5 -> at row:3
True -> at row:5
24.5 -> at row:10
True -> at row:12
24.5 -> at row:17
True -> at row:19
24.5 -> at row:24
True -> at row:26
24.5 -> at row:31
True -> at row:33
24.5 -> at row:38
True -> at row:40


In [104]:
#functionalized.  usage: num_fixer(df, 'name_col')
def num_fixer(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            int(float(row)) # if no error, drop it in the bottom else:'
        except ValueError:
            if row=='True' or row=='False': #drop if would-be boolean
                data_frame.drop([cnt], inplace=True) 
            elif row=='nan':
                data_frame.loc[cnt, series_name] = np.nan
            else: #Chars or NaNs
                pass
        else:
            data_frame.drop([cnt], inplace=True) 
        cnt+=1
    data_frame[series_name] = data_frame[series_name].astype('string', errors='raise')
    data_frame.reset_index(drop=True, inplace=True)  

In [105]:
num_fixer(df,'name_col')

In [106]:
char_fixer(df,'num_col')

In [117]:
df.head(10)

,num_col,name_col
0,3.00,bob
1,5.46,sue
2,62.10,steve
3,3.00,<NA>
4,5.46,bob
5,62.10,sue
6,NaN,steve
7,5.46,<NA>
8,62.10,bob
9,NaN,sue


In [108]:
df.dtypes

num_col     float64
name_col     string
dtype: object

In [109]:
df.isnull().sum()

num_col     5
name_col    4
dtype: int64

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   num_col   15 non-null     float64
 1   name_col  16 non-null     string 
dtypes: float64(1), string(1)
memory usage: 448.0 bytes


Make a version where we don't drop anything.... all go to NaNs

In [87]:
#functionalized.  usage: num_fixer_nan(df, 'name_col')
def num_fixer_nan(data_frame, series_name):
    cnt=0
    for row in data_frame[series_name]:
        try:
            int(float(row)) # if no error, drop it in the bottom else:'
        except ValueError:
            if row=='True' or row=='False': #drop if would-be boolean
                data_frame.loc[cnt, series_name] = np.nan
            elif row=='nan':
                data_frame.loc[cnt, series_name] = np.nan
            else: #Chars or NaNs
                pass
        else:
            data_frame.loc[cnt, series_name] = np.nan
        cnt+=1
    data_frame[series_name] = data_frame[series_name].astype('string', errors='raise')
    data_frame.reset_index(drop=True, inplace=True)

In [22]:
num_fixer_nan(df,'name_col')

In [23]:
df

,num_col,name_col
0,3,bob
1,5.46,sue
2,62.1,steve
3,nan,<NA>
4,False,kim
...,...,...
37,5.46,steve
38,62.1,<NA>
39,nan,kim
40,False,<NA>


In [24]:
df.dropna(inplace=True)

In [25]:
df['name_col'].isnull().values.sum()

0

In [26]:
df

,num_col,name_col
0,3,bob
1,5.46,sue
2,62.1,steve
4,False,kim
6,3,nan
...,...,...
35,angie,bob
36,3,sue
37,5.46,steve
39,nan,kim


## Speed of functions

Testing on fixer function as %timeit runs 1000s of loops and finder functions print out data

In [67]:
dfbig = pd.DataFrame({'name_col' : np.tile(['bob', 'sue', 'steve', 24.5, 'kim', True, np.nan], 6000),
                      'num_col'  : np.tile([3, 5.46, 62.1, np.nan, False, 'angie'], 7000)})

In [64]:
dfbig.shape

(42000, 2)

In [65]:
%timeit  char_fixer(dfbig,'num_col')

9.74 ms ± 1.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Try to be faster by using pandas apply() instead

In [66]:
def char_by_line_fixer(element):
    try:
        float(element) #changed to float to not flag NaNs or decimals.  Adjust as necessary
        return element
    except ValueError:
        return np.nan

In [ ]:
dfbig['num_col'] = dfbig['num_col'].apply(char_by_line_fixer)

In [68]:
%timeit dfbig['num_col'] = dfbig['num_col'].apply(char_by_line_fixer)

22.6 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
char_finder(dfbig, 'num_col')

num_col


In [29]:
dfbig.head(7)

,name_col,num_col
0,bob,3.00
1,sue,5.46
2,steve,62.10
3,24.5,NaN
4,nan,3.00
5,bob,5.46
6,sue,62.10
